In [26]:
#!pip install yfinance

In [27]:
#!pip install plotly

In [28]:
#!pip install --upgrade nbformat

In [29]:
# load libraries
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import warnings
warnings.filterwarnings('ignore')
pd.options.display.float_format = '${:,.2f}'.format

# Settings

In [30]:
symbol = 'BTC-USD'
#symbol = 'ETH-USD'

# Import

In [31]:
# get data
today = datetime.today().strftime('%Y-%m-%d')
start_date = '2016-01-01'
eth_df = yf.download(symbol, start_date, today)
eth_df.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-01-04,"$46,458.85","$47,406.55","$45,752.46","$45,897.57","$45,897.57",42494677905
2022-01-05,"$45,899.36","$46,929.05","$42,798.22","$43,569.00","$43,569.00",36851084859
2022-01-06,"$43,565.51","$43,748.72","$42,645.54","$43,160.93","$43,160.93",30208048289
2022-01-07,"$43,153.57","$43,153.57","$41,077.45","$41,557.90","$41,557.90",84196607520
2022-01-08,"$41,657.84","$42,179.82","$40,702.82","$41,039.96","$41,039.96",30816555008


# Analyse

In [32]:
# analysis
eth_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2200 entries, 2016-01-01 to 2022-01-08
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       2200 non-null   float64
 1   High       2200 non-null   float64
 2   Low        2200 non-null   float64
 3   Close      2200 non-null   float64
 4   Adj Close  2200 non-null   float64
 5   Volume     2200 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 120.3 KB


In [33]:
eth_df.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [34]:
eth_df.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

# Transform

In [35]:
# We’ll reset the index, and we can have our Date as a column.
eth_df.reset_index(inplace=True)
eth_df.columns 

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [36]:
df = eth_df[["Date", "Open"]]
new_names = {
    "Date": "ds", 
    "Open": "y",
}
df.rename(columns=new_names, inplace=True)

In [37]:
df.tail()

,ds,y
2195,2022-01-04,"$46,458.85"
2196,2022-01-05,"$45,899.36"
2197,2022-01-06,"$43,565.51"
2198,2022-01-07,"$43,153.57"
2199,2022-01-08,"$41,657.84"


In [38]:
# plot the open price
x = df["ds"]
y = df["y"]
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y))
# Set title
fig.update_layout(
    title_text="Time series plot of " + symbol + " Open Price",
)

# Model

In [39]:
m = Prophet(
    seasonality_mode="multiplicative" 
)
m.fit(df)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [40]:
future = m.make_future_dataframe(periods = 10, include_history=False)
future

,ds
0,2022-01-09
1,2022-01-10
2,2022-01-11
3,2022-01-12
4,2022-01-13
5,2022-01-14
6,2022-01-15
7,2022-01-16
8,2022-01-17
9,2022-01-18


In [41]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

,ds,yhat,yhat_lower,yhat_upper
0,2022-01-09,"$63,670.59","$60,342.80","$66,650.00"
1,2022-01-10,"$65,141.41","$62,039.57","$68,735.30"
2,2022-01-11,"$66,402.56","$63,112.09","$69,729.34"
3,2022-01-12,"$67,458.74","$64,135.94","$70,584.72"
4,2022-01-13,"$68,997.07","$65,703.79","$72,128.81"
5,2022-01-14,"$70,121.52","$67,032.73","$73,524.06"
6,2022-01-15,"$71,459.00","$68,225.08","$74,598.57"
7,2022-01-16,"$73,515.78","$70,161.88","$76,747.32"
8,2022-01-17,"$74,893.70","$71,771.89","$77,877.82"
9,2022-01-18,"$76,006.02","$72,731.77","$79,140.70"


In [42]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
forecast[forecast['ds'] == next_day]['yhat'].item()

63670.59302483321

In [43]:
plot_plotly(m, forecast)

In [44]:
plot_components_plotly(m, forecast)